In [1]:
# initializing email handling libraries
import smtplib
import imaplib
import email
from email.mime.text import MIMEText

# initializing web parsers
import urllib.request
from bs4 import *

# initializing handy libs
import datetime
import time
import os

In [2]:
# a timezone class to represent the real timezone of the Packtpub server

class UTC(datetime.tzinfo):
    
    def utcoffset(self, dt):
        return datetime.timedelta(0)

    def tzname(self, dt):
        return "UTC"

    def dst(self, dt):
        return datetime.timedelta(0)

In [3]:
# important constants
months = 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
url = 'https://www.packtpub.com/packt/offers/free-learning'
Name = 'Packt Alert'
Sender = 'packt.alert'
Password = 'Packtpub2018'
Footer = '''

________________________________
Download for free ONLY TODAY at: https://www.packtpub.com/packt/offers/free-learning
'''

wlcm = '''
Packt Alert (c) an UNOFFICIAL automated e-mail alerting system
Packt Alert is in no way linked to the Packt Publishing Ltd. or any of its subsidiaries or owners.
It is solely an independent, non-commercial project aimed at promoting this wonderful initiative.

Send a reply e-mail with any of the following words in the message content to:

subscribe - to add your e-mail address - DONE
unsubscribe - to remove your e-mail - DONE
check - to receive the current freebie info - DONE
instruction - to receive a detailed instruction on how to claim freebies - PENDING
help - to submit a bug, problem or malfunction of the Packt Alert - PENDING

'''

log_dir = 'logs'
sub_dir = 'subs'

if not os.path.exists(log_dir):
    os.mkdir(log_dir)

if not os.path.exists(sub_dir):
    os.mkdir(sub_dir)

In [4]:
# extract the freebie data
def extract_freebie_data():
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    data = soup.body.find_all('div', attrs = {'class': 'dotd-main-book cf'})
    hyperlink = 'https://www.packtpub.com'+data[0].a['href']
    title = data[0].h2.text.strip()
    desc = data[0].div.find('div', attrs = {'class': 'dotd-main-book-summary float-left'}).find_all('div')[2].text.strip()
    return title, hyperlink, desc

In [5]:
# function to log events into daily files
def make_log(text_message):
    t = datetime.datetime.now(UTC())
    time_stamp = f'{t.year}-{t.month:02}-{t.day:02} {t.hour:02}:{t.minute:02}:{t.second:02}'
    file_name = f'{log_dir}/{t.year}-{t.month:02}-{t.day:02}-logfile.txt'
    try:
        with open(file_name, 'a+') as f:
            f.write(time_stamp + ' ' + text_message + '\n')
    except FileNotFoundError:
        with open(file_name, 'w+') as f:
            f.write(time_stamp + ' ' + text_message + '\n')

In [6]:
def send_mail(receiver, message):
    with smtplib.SMTP('smtp.gmail.com', 587) as smtpserver:
        smtpserver.ehlo()
        smtpserver.starttls()
        smtpserver.login(Sender, Password)
        smtpserver.sendmail(Sender, receiver, message)

In [7]:
# sends on-demand daily freebie info
def send_check(receiver):
    t = datetime.datetime.now(UTC())
    d = months[t.month-1] + ' ' + str(t.day) + ' - '
    title, hyperlink, desc = extract_freebie_data()
    body = d + title + '\n' + hyperlink + '\n\n' + desc + Footer
    subject = f'Packtpub Alert - {t.year}-{t.month:02}-{t.day:02} - {title}'

    message = MIMEText(body)
    message['Subject'] = subject
    message['From'] = Name
    try:
        send_mail(receiver, message.as_string())
        make_log(f'Sent CHECK to {receiver}\n')
    except Exception as e:
        make_log(f'FAILED to send CHECK to {receiver}\n{e}\n')

In [8]:
# checks if the subscriber is already on the list and if not, adds him
def add_subscriber(new_sub):
    s = False
    t = datetime.datetime.now(UTC())
    file_name = f'{sub_dir}/{t.year}-{t.month:02}-{t.day:02}-subscribers.txt'
    try:
        with open(file_name, 'r+') as f:
            subs = f.readlines()
    except:
        with open(file_name, 'w+') as f:
            f.write(new_sub + '\n')
            f.seek(0)
            subs = f.readlines()
    
    for sub in subs:
        if sub.lower().strip() == new_sub.lower():
            body = "Your e-mail is already on the subscribers' list!" + Footer
            message = MIMEText(body)
            message['Subject'] = 'Duplicate subscription'
            message['From'] = Name
            send_mail(new_sub, message.as_string())
            make_log(f'DUPLICATE subscription attempt from {new_sub}\n')
            s = True
    
    if not s:
        with open(file_name, 'a+') as f:
            f.write(new_sub+'\n')
        body = "Welcome to the subscribers' list!" + wlcm
        message = MIMEText(body)
        message['Subject'] = 'Welcome to Pack Alert!'
        message['From'] = Name
        send_mail(new_sub, message.as_string())
        make_log(f'SUBSCRIBE attempt successful from {new_sub}')
        send_check(new_sub)

In [9]:
def remove_subscriber(new_sub):
    s = False
    t = datetime.datetime.now(UTC())
    
    file_name = f'{sub_dir}/{t.year}-{t.month:02}-{t.day:02}-subscribers.txt'
    try:
        with open(file_name, 'r+') as f:
            subs = f.readlines()
    except:
        with open(file_name, 'w+') as f:
            f.write(new_sub + '\n')
            f.seek(0)
            subs = f.readlines()
    
    with open(file_name, 'w+') as f:
        for sub in subs:
            if sub.lower().strip() == new_sub.lower():
                body = "Your e-mail was successfully removed from the subscribers' list! Thank you for using Pack Alert."
                message = MIMEText(body)
                message['Subject'] = 'Unsubscribe successful'
                message['From'] = Name
                send_mail(new_sub, message.as_string())
                make_log(f'UNSUBSCRIBE done by {new_sub}\n')
                s = True
            else:
                f.write(sub)
    
    if not s:
        body = "Your e-mail was not found on the subscribers' list! Please use 'subscribe' to join in!"
        message = MIMEText(body)
        message['Subject'] = 'Unsubscribe unsuccessful'
        message['From'] = Name
        send_mail(new_sub, message.as_string())
        make_log(f'UNSUBSCRIBE unsuccessful from {new_sub}\n')

In [10]:
def get_mail_parts(m): # returns local_date, local_message_date, from, to, subject, body
    _, d = imapserver.fetch(m, '(RFC822)')
    raw_email = d[0][1].decode('utf-8')
    email_message = email.message_from_string(raw_email)
    
    email_date = email.utils.parsedate_tz(email_message['Date'])
    if email_date:
        local_date = datetime.datetime.fromtimestamp(email.utils.mktime_tz(email_date))
        local_message_date = str(local_date.strftime("%a, %d %b %Y %H:%M:%S"))
    
    email_from = str(email.header.make_header(email.header.decode_header(email_message['From'])))
    email_to = str(email.header.make_header(email.header.decode_header(email_message['To'])))
    email_subject = str(email.header.make_header(email.header.decode_header(email_message['Subject'])))
    
    for part in email_message.walk():
        if part.get_content_type() == "text/plain":
            email_body = part.get_payload(decode=True).decode('utf-8')
        else:
            continue

    return local_date, local_message_date, email_from, email_to, email_subject, email_body

In [11]:
def alert_mode():
    with imaplib.IMAP4_SSL('imap.gmail.com', 993) as imapserver:
        imapserver.login(Sender, Password)
        imapserver.select('inbox')
        typ, data = imapserver.search(None, 'UNSEEN')
        id_list = data[0].split()
        for m in id_list:
            _, d = imapserver.fetch(m, '(RFC822)')
            raw_email = d[0][1].decode('utf-8')
            email_message = email.message_from_string(raw_email)

            email_date = email.utils.parsedate_tz(email_message['Date'])
            if email_date:
                local_date = datetime.datetime.fromtimestamp(email.utils.mktime_tz(email_date))
                local_message_date = str(local_date.strftime("%a, %d %b %Y %H:%M:%S"))

                email_from = str(email.header.make_header(email.header.decode_header(email_message['From'])))

            for part in email_message.walk():
                if part.get_content_type() == "text/plain":
                    email_body = part.get_payload(decode=True).decode('utf-8')
                else:
                    continue

            command = email_body.lower().strip()
            if command.find('check') != -1:
                send_check(email_from)
            elif command.find('unsubscribe') != -1:
                remove_subscriber(email_from)
            elif command.find('subscribe') != -1:
                add_subscriber(email_from)
            elif command.find('stats') != -1:
                pass
            elif command.find('help') != -1:
                pass
        imapserver.logout()

In [12]:
def new_day_init():
    nd = False
    today = datetime.datetime.now(UTC())
    yesterday = today - datetime.timedelta(1)
    file_name_yesterday = f'{sub_dir}/{yesterday.year}-{yesterday.month:02}-{yesterday.day:02}-subscribers.txt'
    file_name_today = f'{sub_dir}/{today.year}-{today.month:02}-{today.day:02}-subscribers.txt'
    with open(file_name_yesterday) as f:
        subs = f.readlines()
    with open(file_name_today, 'w+') as f:
        f.writelines(subs)
    
    make_log('NEW day initialized\n')

    t = datetime.datetime.now(UTC())
    d = months[t.month-1] + ' ' + str(t.day) + ' - '
    title, hyperlink, desc = extract_freebie_data()
    body = d + title + '\n' + hyperlink + '\n\n' + desc + Footer
    subject = f'Packtpub Alert - {t.year}-{t.month:02}-{t.day:02} - {title}'

    message = MIMEText(body)
    message['Subject'] = subject
    message['From'] = Name

    
    with smtplib.SMTP('smtp.gmail.com', 587) as smtpserver:
        smtpserver.ehlo()
        smtpserver.starttls()
        smtpserver.login(Sender, Password)
        for sub in subs:
            try:
                smtpserver.sendmail(Sender, sub.strip(), message.as_string())
                make_log(f'ALERT SENT to {sub}')
            except Exception as e:
                make_log(f'ALERT FAILED to {sub}\n{e}\n')

In [13]:
datetime.datetime.now(UTC()).day

17

In [15]:
while True:
    t1, h1, d1 = extract_freebie_data() # saves the current freebie data
    today = datetime.datetime.now(UTC()) # saves the current datetime (tz: UTC) 
    time_check = today # temporary attribution
    while time_check.day == today.day: # constantly checking (every 15 sec) for new commands during the day
        alert_mode()
        time.sleep(15)
        time_check = datetime.datetime.now(UTC()) 
    t2, h2, d2 = extract_freebie_data() # if time_check not passed (new day), check if freebie data changed already
    while t1 == t2: # if not, keep checking constantly, every 5 minutes
        t2, h2, d2 = extract_freebie_data()
        time.sleep(300)
    alert_mode() # once more take all commands which might have come in during the checkup
    new_day_init() # initialize a new day

KeyboardInterrupt: 

In [16]:
new_day_init()